In [1]:
import pyspark as ps

spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName("df lecture") \
            .getOrCreate()
sc=spark.sparkContext

In [2]:
dir_link = '/home/asus/spark-dfs/'
yelp_business_df = spark.read.json(dir_link + 'data/yelp_academic_dataset_business.json.gz')

In [4]:
yelp_business_df.show()

+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+----------+-----------+--------------------+-------------+-----+------------+-----+-----+--------+
|          attributes|         business_id|          categories|       city|        full_address|               hours|  latitude|  longitude|                name|neighborhoods| open|review_count|stars|state|    type|
+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+----------+-----------+--------------------+-------------+-----+------------+-----+-----+--------+
|[,,,,,,,, true,,,...|vcNAWiLM4dR7D2nww...|[Doctors, Health ...|    Phoenix|4840 E Indian Sch...|[[17:00, 08:00], ...| 33.499313|-111.983758|   Eric Goldberg, MD|           []| true|           9|  3.5|   AZ|business|
|[true,,,,,,,,,,,,...|UsFtqoBl7naz8AVUB...|         [Nightlife]| Dravosburg|202 McClure St
Dr...|            [,,,,,,]| 40.350519|  -

In [6]:
yelp_business_df.printSchema()

root
 |-- attributes: struct (nullable = true)
 |    |-- Accepts Credit Cards: string (nullable = true)
 |    |-- Accepts Insurance: boolean (nullable = true)
 |    |-- Ages Allowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: struct (nullable = true)
 |    |    |-- casual: boolean (nullable = true)
 |    |    |-- classy: boolean (nullable = true)
 |    |    |-- divey: boolean (nullable = true)
 |    |    |-- hipster: boolean (nullable = true)
 |    |    |-- intimate: boolean (nullable = true)
 |    |    |-- romantic: boolean (nullable = true)
 |    |    |-- touristy: boolean (nullable = true)
 |    |    |-- trendy: boolean (nullable = true)
 |    |    |-- upscale: boolean (nullable = true)
 |    |-- Attire: string (nullable = true)
 |    |-- BYOB: boolean (nullable = true)
 |    |-- BYOB/Corkage: string (nullable = true)
 |    |-- By Appointment Only: boolean (nullable = true)
 |    |-- Caters: boolean (nullable = true)
 |    |-- Coat Check

In [8]:
yelp_business = yelp_business_df.createOrReplaceTempView("yelp_business")

In [9]:
result = spark.sql("SELECT name, city, state,stars, attributes FROM yelp_business LIMIT 10")
result.show()

+--------------------+-----------+-----+-----+--------------------+
|                name|       city|state|stars|          attributes|
+--------------------+-----------+-----+-----+--------------------+
|   Eric Goldberg, MD|    Phoenix|   AZ|  3.5|[,,,,,,,, true,,,...|
|        Clancy's Pub| Dravosburg|   PA|  3.5|[true,,,,,,,,,,,,...|
|Cool Springs Golf...|Bethel Park|   PA|  2.5|[,,,,,,,,,,,,,,,,...|
|    Verizon Wireless| Pittsburgh|   PA|  3.5|[,,,,,,,,,,,,,,,,...|
|       Emil's Lounge|   Braddock|   PA|  4.5|[true,,, full_bar...|
|Alexion's Bar & G...|   Carnegie|   PA|  4.0|[true,,, full_bar...|
|Flynn's E W Tire ...|   Carnegie|   PA|  1.5|[,,,,,,,,,,,,,,,,...|
|Forsythe Miniatur...|   Carnegie|   PA|  4.0|[,,,,,,,,,,,,,,,,...|
|Quaker State Cons...|   Carnegie|   PA|  2.5|[,,,,,,,,,,,,,,,,...|
|Kings Family Rest...|   Carnegie|   PA|  3.5|[true,,, none,, c...|
+--------------------+-----------+-----+-----+--------------------+



In [10]:
name = spark.sql(''' 
SELECT name, stars, city,categories 
FROM yelp_business LATERAL VIEW explode(categories) yelp_business as Ybiz
WHERE (city == 'Phoenix' 
AND stars == 5
AND attributes.`Accepts Credit Cards` =='true'
AND (Ybiz == 'Restaurants' OR Ybiz =='Food'))
''')

In [11]:
name.show()

+--------------------+-----+-------+--------------------+
|                name|stars|   city|          categories|
+--------------------+-----+-------+--------------------+
|     Sunshine Market|  5.0|Phoenix|     [Food, Grocery]|
|       See's Candies|  5.0|Phoenix|[Food, Specialty ...|
|The Water Connection|  5.0|Phoenix|[Food, Ice Cream ...|
|Hostess Bakery Ou...|  5.0|Phoenix|    [Bakeries, Food]|
|    Cave Creek Wines|  5.0|Phoenix|[Food, Beer, Wine...|
|Water and Ice Ahw...|  5.0|Phoenix|[Food, Ice Cream ...|
|       Auslers Grill|  5.0|Phoenix|       [Restaurants]|
|Mulligan's Restau...|  5.0|Phoenix|       [Restaurants]|
|            QuikTrip|  5.0|Phoenix|[Food, Automotive...|
|     Danzeisen Dairy|  5.0|Phoenix|         [Food, CSA]|
|             Sunfare|  5.0|Phoenix|[Food Delivery Se...|
|             Sunfare|  5.0|Phoenix|[Food Delivery Se...|
|        Trader Joe's|  5.0|Phoenix|[Food, Beer, Wine...|
|       Dream Dinners|  5.0|Phoenix|[Food, Do-It-Your...|
|             

In [12]:
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, col

In [13]:
schema = StructType([
    StructField('id',IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('email', StringType(), True),
    StructField('number', StringType(), True)])

In [14]:
yelp = spark.read.csv(dir_link + 'data/users.txt',sep = ';', schema=schema)

In [105]:
yelp.createOrReplaceTempView('users')

In [15]:
yelp.show(5)

+----------+-----------------+--------------------+--------------------+
|        id|             name|               email|              number|
+----------+-----------------+--------------------+--------------------+
|1106214172|Prometheus Barwis|prometheus.barwis...|      (533) 072-2779|
| 527133132|Ashraf Bainbridge|ashraf.bainbridge...|                null|
|1290614884|   Alain Hennesey|alain.hennesey@fa...|(942) 208-8460,(8...|
|1700818057| Hamed Fingerhuth|hamed.fingerhuth@...|                null|
|  17378782|    Annamae Leyte|annamae.leyte@msn...|                null|
+----------+-----------------+--------------------+--------------------+
only showing top 5 rows



In [16]:
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, col


In [96]:
def casting_function(row):
    user_id, money, date = row
    return (int(user_id), float(money), str(date))


transactions_rdd = sc.textFile(dir_link + 'data/transactions.txt')\
                    .map(lambda rowstr : rowstr.replace('$',''))\
                    .map(lambda line : line.split(';'))\
                    .map(casting_function)
                    #.map(lambda line : casting_function(line))
transactions_rdd.collect()

[(815581247, 144.82, '2015-09-05'),
 (1534673027, 140.93, '2014-03-11'),
 (842468364, 104.26, '2014-05-06'),
 (1720001139, 194.6, '2015-08-24'),
 (1397891675, 307.72, '2015-09-25'),
 (926282663, 36.69, '2014-10-24'),
 (694853136, 39.59, '2014-11-26'),
 (636287877, 430.94, '2015-06-12'),
 (1396310477, 31.4, '2014-12-05'),
 (1279939289, 180.69, '2015-03-26'),
 (859061953, 383.35, '2014-06-06'),
 (1983919868, 256.2, '2015-09-28'),
 (589339046, 930.56, '2014-09-21'),
 (1559785598, 423.77, '2015-05-18'),
 (347589978, 309.53, '2015-10-11'),
 (963722938, 299.19, '2014-04-06'),
 (1808365853, 426.21, '2015-09-10'),
 (417552135, 732.27, '2015-09-30'),
 (744965566, 186.33, '2015-12-30'),
 (1513020241, 925.8, '2014-10-06'),
 (1053481166, 679.98, '2015-01-02'),
 (1796495372, 569.92, '2015-12-19'),
 (175508097, 335.88, '2014-09-10'),
 (1961899399, 818.2, '2015-05-12'),
 (1310696499, 422.99, '2015-08-10'),
 (1481791532, 759.72, '2015-06-28'),
 (1279296600, 789.16, '2015-08-03'),
 (296132510, 976.36, 

In [97]:
casting_function(['815581247','144.82','2015-09-05'])

(815581247, 144.82, '2015-09-05')

In [101]:
schema = StructType([
    StructField('user_id',IntegerType(), True),
    StructField('money', FloatType(), True),
    StructField('date', StringType(), True)])

df = spark.createDataFrame(transactions_rdd, schema)

# show the result
df.show()

# print the schema
df.printSchema()

df = df.select('user_id', unix_timestamp('date', 'yyyy-MM-dd').cast(TimestampType()).cast(DateType()).alias('proper_date'),'money')
df.show()
df.printSchema()

+----------+------+----------+
|   user_id| money|      date|
+----------+------+----------+
| 815581247|144.82|2015-09-05|
|1534673027|140.93|2014-03-11|
| 842468364|104.26|2014-05-06|
|1720001139| 194.6|2015-08-24|
|1397891675|307.72|2015-09-25|
| 926282663| 36.69|2014-10-24|
| 694853136| 39.59|2014-11-26|
| 636287877|430.94|2015-06-12|
|1396310477|  31.4|2014-12-05|
|1279939289|180.69|2015-03-26|
| 859061953|383.35|2014-06-06|
|1983919868| 256.2|2015-09-28|
| 589339046|930.56|2014-09-21|
|1559785598|423.77|2015-05-18|
| 347589978|309.53|2015-10-11|
| 963722938|299.19|2014-04-06|
|1808365853|426.21|2015-09-10|
| 417552135|732.27|2015-09-30|
| 744965566|186.33|2015-12-30|
|1513020241| 925.8|2014-10-06|
+----------+------+----------+
only showing top 20 rows

root
 |-- user_id: integer (nullable = true)
 |-- money: float (nullable = true)
 |-- date: string (nullable = true)

+----------+-----------+------+
|   user_id|proper_date| money|
+----------+-----------+------+
| 815581247| 201

In [102]:
df.createOrReplaceTempView('UserMoney')

In [164]:
last = spark.sql('''
SELECT id, SUM(UserMoney.money), users.name
FROM UserMoney, users
WHERE id = user_id
GROUP BY id, user_id, users.name
ORDER BY sum(money) DESC
''')
last.show()

+----------+------------------+--------------------+
|        id|        sum(money)|                name|
+----------+------------------+--------------------+
| 196163569|21945.299961090088|  Kashawn Macpherson|
| 681405035|21773.509883880615|       Brysten Jeffs|
|1105929083|21120.549940109253|      Martez Carlyle|
| 171453878|20641.110204696655|         Jaivyn Hoks|
|1295065819| 20380.15988922119|       Bryanne Stopp|
| 791379582|20322.109855651855|Leanthony Waldegrave|
| 488297509| 20230.06000518799| Roosevelt Gooderham|
|1702604948|20172.169887542725|       Demont Howell|
| 707056747| 20163.90997314453|      Nasteha Bister|
|1438177405| 19998.19010066986|    Analaura Beetham|
| 175469806| 19915.26010131836|  Jezabel Lafreniere|
|1356654037|19820.009912490845|     Harmany Slemons|
|1839031433|19629.419967651367|   Alexsandro Bartyn|
| 819593679| 19589.98989391327|  Sophialynn Ramsden|
| 693179410|19588.799982070923|       Andri Fordyce|
| 473705896|  19525.3399848938|        Joia De

In [170]:
# Dataframe Style
new_df = yelp.join(df, df.user_id == yelp.id, how='inner').orderBy('money', ascending=False)\
.select(['name','money', 'id'])

new_df.show(10)

+-----------------+------+----------+
|             name| money|        id|
+-----------------+------+----------+
|      Raziel Merk|999.99| 504736332|
|    Andrian Waite|999.99| 225990677|
|    Kianu Dyneley|999.99| 197275390|
|   Landri Fulshur|999.99|1093225999|
|      Ori Horrage|999.98| 740624030|
|Leilani Cranstoun|999.98|1009490315|
|Samyrah Milbourne|999.98|  50874512|
|   Vishwak Farrow|999.98| 420754422|
|    Veida Hubbard|999.98|2141604701|
|   Zasia Scrivens|999.98|1378643543|
+-----------------+------+----------+
only showing top 10 rows

